In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glitch_this import ImageGlitcher
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from torchvision import datasets, transforms
from torchinfo import summary
import os
import PIL

In [7]:
torch.cuda.is_available()

False

In [2]:
def get_images(path):
    images_list = []
    
    for root, dirs, files in os.walk(path):
        for name in files:
            images_list.append(os.path.join(root, name))

    return images_list

In [3]:
# S_resize = 224
# x = np.linspace(0, 1, S_resize)
# y = np.linspace(0, 1, S_resize)
# X, Y = np.meshgrid(x, y)
# periodic_noise = np.sin(32*np.pi*(X + Y/50))
# result = img + 10*periodic_noise


In [4]:
S_resize = 224
x = np.linspace(0, 1, S_resize)
y = np.linspace(0, 1, S_resize)
X, Y = np.meshgrid(x, y)

def add_periodic_noise(img):
    result = img
    dense = np.random.randint(32,100)
    alpha = np.random.randint(5,30)
    
    if np.random.randint(0,2) == 0:
        a = np.random.randint(1,50)
        b = 1
    else:
        a = 1
        b = np.random.randint(1,50)
    sin_periodic_noise = np.sin(dense*np.pi*(X/a + Y/b))
    result = img + alpha*sin_periodic_noise
    
    if np.random.randint(0,2) == 1:
        alpha_1 = np.random.randint(20,40)
        dense_1 = np.random.randint(25,40)
        result += alpha_1*np.cos(dense_1*np.pi*X)
    return result.astype(np.float32)

In [5]:
class CustomDataset(Dataset):

    def __init__(self, root, target, transform=None):
        self.root = root
        self.images = get_images(root)
        self.targets = [int(target) for i in range(len(self.images))]
        self.transform = transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        image_name = self.images[index]  
        image = PIL.ImageOps.grayscale(PIL.Image.open(image_name))
        label = torch.tensor(self.targets[index],dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return (image, label)

In [6]:
root = "../Data/505_video_frames/"
batch_size=32

normal_transform = transforms.Compose([
        transforms.Resize((S_resize,S_resize)),           
        transforms.ToTensor(),
        transforms.Normalize([0.456],
                             [0.227])
    ])

periodic_transform = transforms.Compose([
        transforms.Resize((S_resize,S_resize)),      
        transforms.Lambda(add_periodic_noise),
        transforms.ToTensor(),
        transforms.Normalize([0.456],
                             [0.227])
    ])

In [7]:
original_dataset = CustomDataset(root=root,target=0,transform=normal_transform)
noised_dataset = CustomDataset(root=root,target=1,transform=periodic_transform)
dataset = ConcatDataset([original_dataset, noised_dataset])

In [8]:
inv_normalize = transforms.Normalize(
    mean=[-0.456/0.227],
    std=[1/0.227]
)

In [9]:
train_size = int(0.6 * len(dataset))
validation_size = int((len(dataset) - train_size)/2)
test_size = int(len(dataset) - train_size - validation_size)
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size, test_size]) #6 2 2

In [10]:
inv_normalize = transforms.Normalize(
    mean=[-0.456/0.227],
    std=[1/0.227]
)

In [11]:
# im_inv = inv_normalize(train_dataset[150][0])
# plt.imshow(np.transpose(im_inv.numpy(), (1, 2, 0)),cmap='gray');

In [12]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [13]:
len(train_loader)

3428

In [14]:
class PeriodicNoiseDetect(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.conv3 = nn.Conv2d(16,32,3,1)
        self.conv4 = nn.Conv2d(32,64,3,1)
        self.conv5 = nn.Conv2d(64,64,3,1)
        self.fc1 = nn.Linear(1600,512)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512,1)  
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x,2,2)

        x = x.view(-1,1600)

        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        return F.sigmoid(x)

In [15]:
model = PeriodicNoiseDetect()

In [16]:
print(summary(model, input_size=(batch_size, 1, 224, 224)))

Layer (type:depth-idx)                   Output Shape              Param #
PeriodicNoiseDetect                      [32, 1]                   --
├─Conv2d: 1-1                            [32, 6, 222, 222]         60
├─Conv2d: 1-2                            [32, 16, 109, 109]        880
├─Conv2d: 1-3                            [32, 32, 52, 52]          4,640
├─Conv2d: 1-4                            [32, 64, 24, 24]          18,496
├─Conv2d: 1-5                            [32, 64, 10, 10]          36,928
├─Linear: 1-6                            [32, 512]                 819,712
├─Dropout: 1-7                           [32, 512]                 --
├─Linear: 1-8                            [32, 1]                   513
Total params: 881,229
Trainable params: 881,229
Non-trainable params: 0
Total mult-adds (G): 1.32
Input size (MB): 6.42
Forward/backward pass size (MB): 157.72
Params size (MB): 3.52
Estimated Total Size (MB): 167.67


In [17]:
class EarlyStopper:
    def __init__(self, patience=2, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss and abs(validation_loss - self.min_validation_loss) > self.min_delta:
            self.counter = 0
            self.min_validation_loss = validation_loss
        else:
            self.counter += 1
            if self.counter >= self.patience and validation_loss < 0.01 :
                return True
        return False

In [18]:
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=0.00001,weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.1,patience=2,verbose=True)
early_stopper = EarlyStopper(patience=3, min_delta=0.1)

In [19]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>8}')
    print(f'________\n{sum(params):>8}')

count_parameters(model)

      54
       6
     864
      16
    4608
      32
   18432
      64
   36864
      64
  819200
     512
     512
       1
________
  881229


In [20]:
from tqdm import tqdm

In [22]:
epochs = 15

# train_losses = []
# test_losses = []

for i in range(epochs):
    for b, (X_train, y_train) in tqdm(enumerate(train_loader)):
        b+=1
        #push data to GPU
        model.train()
        X_train = X_train.to(device='cuda')
        y_train = y_train.to(device='cuda')
        # Apply the model
        pred = model(X_train)
        loss = criterion(pred, y_train.unsqueeze(dim=-1))
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print interim results
        if b % 100 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{batch_size*b:6}/{len(train_loader)}]  loss: {loss.item()}')
        
        if early_stopper.early_stop(loss):             
            break
    # train_losses.append(loss.item())

    running_loss = 0    
    model.eval()
    with torch.no_grad():
        for vb, (X_test, y_test) in enumerate(test_loader):
            #push data to GPU
            X_test = X_test.cuda()
            y_test = y_test.cuda()
            # Apply the model
            val = model(X_test)
            loss = criterion(val, y_test.unsqueeze(dim=-1))
            running_loss += loss
    
    # test_losses.append(loss)
    avg_loss = running_loss / (vb+1)
    print(f'epoch: {i:2}  finished   validation loss: {avg_loss.item():10.8f}')
    scheduler.step(loss)

0it [00:00, ?it/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    for X_test, y_test in tqdm(test_loader):
        X_test = X_test.to(device='cuda')
        y_test = y_test.to(device='cuda')
        y_val = model(X_test)
        predicted = torch.round(y_val)
        correct += (predicted == y_test.unsqueeze(dim=-1)).sum()
print(f'Test accuracy: {correct.item()}/{len(test_dataset)} = {correct.item()*100/len(test_dataset):7.3f}%')

In [ ]:
new_sample_path = '../castle.jpg'
new_sample = PIL.ImageOps.grayscale(PIL.Image.open(new_sample_path))
# new_sample = new_sample.resize((S_resize,S_resize))
# # new_sample = add_periodic_noise(new_sample)
# new_sample += 30*np.cos(100*np.pi*X)


plt.imshow(new_sample,cmap='gray')
# new_sample = PIL.Image.fromarray(np.uint8(new_sample)).convert('L')

In [ ]:
img = normal_transform(new_sample)

In [ ]:
inv_normalize = transforms.Normalize(
    mean=[-0.456/0.227],
    std=[1/0.227]
)
im_inv = inv_normalize(img)
plt.imshow(np.transpose(im_inv.numpy(), (1, 2, 0)),cmap='gray');

In [ ]:
model.eval()
with torch.no_grad():
    img = img.unsqueeze(dim=0)
    print(img.size())
    print(model(img.to(device='cuda')))

In [ ]:
img.size()